In [ ]:
# Set up variables for agents
# Create a new GCP Project
# Run gcloud init if needed
import os

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

In [ ]:
# Enable APIs
!gcloud services enable firestore.googleapis.com aiplatform.googleapis.com logging.googleapis.com storage.googleapis.com monitoring.googleapis.com cloudtrace.googleapis.com telemetry.googleapis.com

In [ ]:
# Set up dataebase. Only run once.
!gcloud firestore databases create --location=nam5 --database='inventory' --quiet

In [ ]:
# Install requirements if haven't done so yet. 
%pip install -r agent/requirements.txt

In [ ]:
# Run locally
!adk web

# Example Prompts:
# What items do I have?
# Add Inception 4k New
# What is the value of my items?
# * Find a picture of a blu ray disk and upload it to the agent to add it to inventory.

In [ ]:
# Deploy agents to GCP

!adk deploy agent_engine agent --project=$PROJECT_ID --agent_engine_config_file=agent/.agent_engine_config.json

In [ ]:
# Provide Agent access to Firestore
# Quick hack(?) to update Service Account to have access to firestore database

PROJECT_NUMBER = !gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NUMBER = PROJECT_NUMBER[0]
print(PROJECT_NUMBER)

In [ ]:
# We only need to run this once (not sure if ServiceAccount IAM gets created before Agent exists)

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:service-$PROJECT_NUMBER@gcp-sa-aiplatform-re.iam.gserviceaccount.com" \
    --role="roles/datastore.user"

# We may need to redeploy the agent to have access to db

In [ ]:
# Access deployed agent from here or log into Agent Engine -> Playgroun to test it there.

import vertexai
from vertexai import agent_engines

vertexai.init(project=PROJECT_ID, location=LOCATION)

agents_list = list(agent_engines.list())
if agents_list:
    remote_agent = agents_list[0]  # Get the first (most recent) agent
    client = agent_engines
    print(f"✅ Connected to deployed agent: {remote_agent.resource_name}")
else:
    print("❌ No agents found. Please deploy first.")

In [ ]:
async for item in remote_agent.async_stream_query(
    message="What items do I have?",
    user_id="user_42",
):
    print(item)

In [ ]:
agent_engines.delete(resource_name=remote_agent.resource_name, force=True)

print("✅ Agent successfully deleted")